Data Exploration
=======
In this section is mainly to show the raw data structure, cleaning and vectorization process. With the help of `parw` and

In [22]:
import os
import pandas as pd

os.chdir('D:\\github\\master_thesis_2022')
cwd = os.getcwd()

In [23]:
raw = pd.read_csv(os.path.join(cwd, 'data\\interim\\df_raw.csv'), encoding='utf-8-sig')
raw.head()

,id,url,title,score,num_comments,created_utc,selftext,top_comments
0,tu2rj7,https://www.reddit.com/r/wallstreetbets/commen...,r/place,1,2,1.648850e+09,[removed],NaN
1,tu2mtj,https://i.redd.it/yp2zfi16ozq81.jpg,Diamond hands in Madison WI,1,0,1.648850e+09,NaN,NaN
2,tu2mmh,https://www.reddit.com/r/wallstreetbets/commen...,GameStop's Board of Directors wants to dilute ...,0,12,1.648850e+09,The below is an excerpt from GameStop's recent...,1 Billion shares for a mall/strip-mall brick-a...
3,tu2men,https://www.reddit.com/r/wallstreetbets/commen...,ok..OK... what happened today?,0,24,1.648850e+09,[removed],"dude its a momentum trade, theres no real sens..."
4,tu2do4,https://i.redd.it/xtiyaju5mzq81.jpg,Just a couple of my positions… ($SPY & $NIO hi...,1,2,1.648849e+09,NaN,NaN


In [24]:
clean = pd.read_csv(os.path.join(cwd, 'data\\interim\\df_clean.csv'), encoding='utf-8-sig')
clean.head()

,id,url,title,score,num_comments,created_utc,selftext,top_comments
0,tu2rj7,https://www.reddit.com/r/wallstreetbets/commen...,r/place,1,2,04-01-22,NaN,NaN
1,tu2mtj,https://i.redd.it/yp2zfi16ozq81.jpg,Diamond hands in Madison WI,1,0,04-01-22,NaN,NaN
2,tu2mmh,https://www.reddit.com/r/wallstreetbets/commen...,GameStop's Board of Directors wants to dilute ...,0,12,04-01-22,The below is an excerpt from GameStop's recent...,1 Billion shares for a mall/strip-mall brick-a...
3,tu2men,https://www.reddit.com/r/wallstreetbets/commen...,ok..OK... what happened today?,0,24,04-01-22,NaN,"dude its a momentum trade, theres no real sens..."
4,tu2do4,https://i.redd.it/xtiyaju5mzq81.jpg,Just a couple of my positions… ($SPY & $NIO hi...,1,2,04-01-22,NaN,NaN
